## A - Using TorchText with Your Own Datasets

> Trong Series này, chúng ta đa sử dụng tập dữ liệu IMDb trong TorchText. TorchText có rất nhiều tập dữ liệu kinh điển, phục vụ cho các bài toán như: classification, language modelling, sequence tagging, etc. Tuy nhiên, chúng ta sẽ cần phải sử dụng các dataset không nằm trong TorchText. May mắn là TorchText có các hàm support việc xây dựng dataset từ bộ dữ liệu bên ngoài.

> Có 3 dạng data mà TorchText có thể đọc: `json`, `tsv` và `csv`.

> **Theo ý kiến của tôi, `json` là kiểu dữ liệu phù hợp nhất với TorchText**.

### Reading JSON

> Bắt đầu với định dạng `json`. Hiển nhiên, data cần xử lý phải ở dạng `json`.

> Ta define các trường sau:

In [1]:
from torchtext.legacy import data
from torchtext.legacy import datasets

NAME = data.Field()
PLACE = data.Field()
SAYING = data.Field()

> Tiếp theo, ta cần thông báo cho TorchText trường nào áp dụng cho `json` object nào.

> Đối với `json` data, ta cần tạo một dictionary đáp ứng các tiêu chí sau:
>> * Key của dictionary phải match với key của `json` object.
>> * Value của dictionary là một tupple với:
>>> * Phần tử đầu tiên là tên của batch object's attribute.
>>> * Phần tử thứ 2 là tên của trường `Field`.

> Lý giải cho cụm từ "batch objet's attribute": Quay lại các notebooks trước, ta truy cập vào các trường `TEXT` và `LABEL` trong quá trình train/evaluation bằng cách sử dụng `batch.text` và `batch.label`. Ta truy cập được vào do TorchText thiết lập batch object có attribute `text` và `label`, mỗi attribute là một tensor chứa text và label.

> Dưới đây là 1 số chú ý quan trọng:
>> * Thứ tự của key trong dictionary `fields` không quan trọng, miễn là keys phải match với keys trong `json` data.
>> * Tên của `Field` không nhất thiết phải match với keys trong `json` object. Ví dụ ta có thể sử dụng `PLACE` cho trường `location`.
>> * Khi làm việc với `json` data, không phải lúc nào ta cũng sử dụng hết các keys có trong data.
>> * Nếu values của `json` là string, `Fields` tokenization sẽ được áp dụng lên nó (mặc định sẽ là ngăn cách bởi spaces). Tuy nhiên, nếu values là một list, TorchText sẽ không áp dụng tokenization lên nó. Thông thường, ta nên tokenize data trước vì nó giúp tiết kiệm thời gian do TorchText không cần tokenize nữa.
>> * Value trong các trường của `json` không nhất thiết phải có cùng kiểu. 
>> * Nếu sử dụng trường trong `json`, tất cả các single example phải là instance của trường đó. Trong ví dụ dưới, tất cả các examples phải có name, location và quote. Tuy nhiên, do ta không dùng trường age nên nếu examples thiếu trường age thì cũng không ảnh hưởng gì.

In [2]:
fields = {'name': ('n', NAME), 'location': ('p', PLACE), 'quote': ('s', SAYING)}

> Bây giờ, trong training loop, ta có thể lặp trong iterator và truy cập tên thông qua `batch.n`, location thông qua `batch.p` và quote thông qua `batch.s`.

> Sau đó, ta tự tạo datasets (`train_data` và `test_data`) bằng hàm `TabularDataset.splits`.

In [3]:
train_data, test_data = data.TabularDataset.splits(
    path = './data',
    train = 'train.json',
    test = 'test.json',
    format = 'json',
    fields = fields
)

> Đối số `path` xác định thư mục cha của 2 file data, `train` và `test` arguments là đối số chứa tên của 2 file. ở đây train dataset có đường dẫn: `data/train.json`.

> Ở đây, ta đã thông báo cho function biết đang sử dụng `json` data, và truyền `fields` dictionary đã được định nghĩa từ trước.

> Dưới đây là ví dụ khi ta có sẵn 3 file: train.json, test.json và valid.json.

In [4]:
train_data, test_data, valid_data = data.TabularDataset.splits(
    path = './data',
    train = 'train.json',
    test = 'test.json',
    validation = 'valid.json',
    format = 'json',
    fields = fields
)

> Lưu ý, ở đây các tên trường (field names) là `n`, `p` và `s` khớp với những gì đã được define trong `fields` dictionary.

> Bên cạnh đó, ta cũng cần chú ý: Từ "United Kingdom" trong `p` được tokenize trong khi "united kingdom" trong `s` không được tokenize. Lý do là TorchText sẽ không áp dụng tokenize khi value có kiểu dữ liệu là list.

In [5]:
vars(train_data[0])

{'n': ['John'],
 'p': ['United', 'Kingdom'],
 's': ['i', 'love', 'the', 'united kingdom']}

> Từ bây giờ, ta có thể sử dụng `train_data`, `test_data` và `valid_data` để xây dựng vocabulary và tạo iterators. Ta có thể truy cập vào các attribute bằng cách sử dụng `batch.n`, `batch.p` và `batch.s` để lấy names, places và sayings.

### Reading CSV/TSV

> Tất nhiên, dữ liệu phải ở dạng CSV hoặc TSV.

>Đối với CSV/TSV data, ta sử dụng list of tupple thay vì list như ở `json`. Phần tử đầu tiên trong tupple là tên của batch object's attribute, phần tử thứ hai là tên `Field`.

> Không giống như `json` data, tupples phải có thứ tự giống trong file `csv/tsv`. do đó, khi cần bỏ qua một cột data, ta sử dụng `None`. Tuy nhiên, nếu ta muốn sử dụng các cột dữ liệu liền nhau và đứng đầu tiên, ta chỉ cần dùng 2 tupples.

In [6]:
fields = [('n', NAME), ('p', PLACE), (None, None), ('s', SAYING)]

> Ta chuyển `format` thành `tsv`, set tham số `skip_header = True` nếu data có header.

In [7]:
train_data, test_data, valid_data = data.TabularDataset.splits(
    path = './data',
    train = 'train.csv',
    test = 'test.csv',
    validation = 'valid.csv',
    format = 'csv',
    fields = fields,
    skip_header = True
)

In [8]:
vars(train_data.examples[0])

{'n': ['John'],
 'p': ['United', 'Kingdom'],
 's': ['i', 'love', 'the', 'united', 'kingdom']}

> Tương tự với TSV file.

In [9]:
train_data, test_data, valid_data = data.TabularDataset.splits(
    path = './data',
    train = 'train.tsv',
    test = 'test.tsv',
    validation = 'valid.tsv',
    format = 'tsv',
    fields = fields,
    skip_header = True
)

In [10]:
vars(train_data.examples[0])

{'n': ['John'],
 'p': ['United', 'Kingdom'],
 's': ['i', 'love', 'the', 'united', 'kingdom']}

### Why JSON over CSV/TSV?

1. `csv` hoặc `tsv` data không thể lưu trữ trong lists. Điều đó có nghĩa là data không thể tokenize trước khi đưa vào TorchText, dẫn đến việc TorchText phải tokenize mỗi khi đưa data vào. Việc sử dụng các tokenizer bên ngoài như của `spaCy` tốn khá ít thời gian, do đó, **ta nên tokenize datasets và lưu chúng dưới dạng `json lines`**.
2. Nếu tabs xuất hiện trong `tsv` data, còn commas xuất hiện trong `csv` data, TorchText sẽ nghĩ đó là ngăn cách cột, dẫn đến việc phân tích data bị sai. Do `json` data nằm trong dictionary, ta truy cập chúng thông qua keys nên không cần lo lắng về việc xuất hiện các ký tự ngăn cách. 

### Iterators

> Xây dựng vocabulary cho datasets.

In [11]:
NAME.build_vocab(train_data)
PLACE.build_vocab(train_data)
SAYING.build_vocab(train_data)

> Tiếp theo, ta cần tạo iterators.

> Mặc định, train data được shuffled ở mỗi epoch, còn validation/test data thì được sắp xếp. Tuy nhiên TorchText không biết sắp xếp data nếu ta không yêu cầu nó làm, từ đó có thể gây lỗi.

> Có 2 cách để xử lý:
>> * Truyền `sort = False`.
>> * Sử dụng `sort_key`. `sort_key` nhận vào một hàm trả về key mà ta muốn sắp xếp dữ lệu theo.

In [12]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 1

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort = False, #don't sort test/validation data
    batch_size=BATCH_SIZE,
    device=device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key = lambda x: x.s, #sort by s attribute (quote)
    batch_size=BATCH_SIZE,
    device=device)

print('Train:')
for batch in train_iterator:
    print(batch)
    
print('Valid:')
for batch in valid_iterator:
    print(batch)
    
print('Test:')
for batch in test_iterator:
    print(batch)

Train:

[torchtext.legacy.data.batch.Batch of size 1]
	[.n]:[torch.LongTensor of size 1x1]
	[.p]:[torch.LongTensor of size 2x1]
	[.s]:[torch.LongTensor of size 5x1]

[torchtext.legacy.data.batch.Batch of size 1]
	[.n]:[torch.LongTensor of size 1x1]
	[.p]:[torch.LongTensor of size 2x1]
	[.s]:[torch.LongTensor of size 4x1]
Valid:

[torchtext.legacy.data.batch.Batch of size 1]
	[.n]:[torch.LongTensor of size 1x1]
	[.p]:[torch.LongTensor of size 1x1]
	[.s]:[torch.LongTensor of size 3x1]

[torchtext.legacy.data.batch.Batch of size 1]
	[.n]:[torch.LongTensor of size 1x1]
	[.p]:[torch.LongTensor of size 2x1]
	[.s]:[torch.LongTensor of size 3x1]
Test:

[torchtext.legacy.data.batch.Batch of size 1]
	[.n]:[torch.LongTensor of size 1x1]
	[.p]:[torch.LongTensor of size 1x1]
	[.s]:[torch.LongTensor of size 2x1]

[torchtext.legacy.data.batch.Batch of size 1]
	[.n]:[torch.LongTensor of size 1x1]
	[.p]:[torch.LongTensor of size 1x1]
	[.s]:[torch.LongTensor of size 4x1]


## END